In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
import os
import matplotlib.pyplot as plt

# 设置随机种子以保证结果可复现
torch.manual_seed(42)
np.random.seed(42)

# 1. 加载并预处理数据
def load_and_preprocess_data(file_path, skiprows=1):
    """
    从CSV文件加载并预处理数据。

    参数:
    - file_path (str): CSV文件路径。
    - skiprows (int): 开始时要跳过的行数（默认为1，跳过第一行）。

    返回:
    - pd.DataFrame: 预处理后的数据，日期列设置为索引。
    """
    try:
        # 读取CSV，跳过前skiprows行
        data = pd.read_csv(file_path, skiprows=skiprows, header=0)
        # 假设第一列是日期，解析为日期并设置为索引
        data['Date'] = pd.to_datetime(data.iloc[:, 0], format='%Y/%m/%d')
        data = data.set_index('Date')
        # 排除日期列（第一列）
        data = data.iloc[:, 1:]
        # 选择数值列
        data = data.select_dtypes(include=[np.number])
        # 转换为float32类型
        data = data.astype(np.float32)
        # 检查是否存在NaN值
        if data.isnull().values.any():
            print(f"检测到来自{file_path}的数据中存在NaN值。删除包含NaN的行。")
            data = data.dropna()
        return data
    except Exception as e:
        print(f"加载{file_path}时出错: {e}")
        return pd.DataFrame()  # 出错时返回空DataFrame

# 2. 加载所有数据
all_data_path = 'data/OEL_all.csv'
all_data = load_and_preprocess_data(all_data_path, skiprows=1)
if all_data.empty:
    raise ValueError(f"无法从 '{all_data_path}' 加载数据。请检查文件内容。")

all_columns = all_data.columns.tolist()
print(f"所有数据已加载。样本数量: {len(all_data)}, 通道数量: {len(all_columns)}")

# 3. 构建聚合数据集
def create_aggregated_datasets(all_data):
    """
    基于原始数据创建每日、每周、每月和每年的最大值数据集。

    参数:
    - all_data (pd.DataFrame): 原始数据，日期为索引。

    返回:
    - dict: 包含各层级最大值数据集的字典。
    """
    aggregated_data = {}
    aggregated_data['daily_max'] = all_data.resample('D').max()
    aggregated_data['weekly_max'] = all_data.resample('W').max()
    aggregated_data['monthly_max'] = all_data.resample('M').max()
    aggregated_data['yearly_max'] = all_data.resample('A').max()
    return aggregated_data

aggregated_data = create_aggregated_datasets(all_data)
print("聚合数据集已创建。")
for level, df in aggregated_data.items():
    print(f"{level} 样本数量: {len(df)}")

# 4. 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(all_data)  # 使用all_data拟合缩放器

def scale_dataset(df, scaler):
    """
    使用给定的缩放器对DataFrame进行缩放。

    参数:
    - df (pd.DataFrame): 要缩放的数据。
    - scaler (MinMaxScaler): 已拟合的缩放器。

    返回:
    - pd.DataFrame: 缩放后的数据。
    """
    scaled_array = scaler.transform(df)
    scaled_df = pd.DataFrame(scaled_array, columns=df.columns, index=df.index)
    return scaled_df

# 归一化所有数据集
scaled_all_data = scale_dataset(all_data, scaler)
scaled_daily_max = scale_dataset(aggregated_data['daily_max'], scaler)
scaled_weekly_max = scale_dataset(aggregated_data['weekly_max'], scaler)
scaled_monthly_max = scale_dataset(aggregated_data['monthly_max'], scaler)
scaled_yearly_max = scale_dataset(aggregated_data['yearly_max'], scaler)

print("数据归一化完成。")

# 5. 定义层级编码（独热向量）
level_encodings = {
    'A': [1, 0, 0, 0],
    'B': [0, 1, 0, 0],
    'C': [0, 0, 1, 0],
    'D': [0, 0, 0, 1],
}

num_levels = len(level_encodings)  # 应为4

# 6. 计算每个层级的输入大小
input_sizes = {
    'A': 48 * len(all_columns) + num_levels,  # 48个半小时
    'B': 7 * len(all_columns) + num_levels,   # 7天
    'C': 4 * len(all_columns) + num_levels,   # 4周
    'D': 12 * len(all_columns) + num_levels   # 12个月
}

# 确定最大输入大小
input_size = max(input_sizes.values())  # 15076
print(f"确定的最大输入大小: {input_size}")

# 7. 计算每个层级输入数据的均值用于填充
mean_values = {}
mean_values['A'] = scaled_all_data.mean().values  # Level A: all_data
mean_values['B'] = scaled_daily_max.mean().values  # Level B: daily_max
mean_values['C'] = scaled_weekly_max.mean().values  # Level C: weekly_max
mean_values['D'] = scaled_monthly_max.mean().values  # Level D: monthly_max

# 8. 定义 Positional Encoding 类
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: (seq_len, batch_size, d_model)
        x = x + self.pe[:x.size(0), :]
        return x

# 9. 定义基于 Transformer 的 Encoder 和 Decoders
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_size, latent_size, num_layers=2, nhead=4, dim_feedforward=256, dropout=0.1):
        super(TransformerEncoderModel, self).__init__()
        self.input_proj = nn.Linear(input_size, dim_feedforward)
        self.positional_encoding = PositionalEncoding(dim_feedforward)
        encoder_layer = nn.TransformerEncoderLayer(d_model=dim_feedforward, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_proj = nn.Linear(dim_feedforward, latent_size)

    def forward(self, x):
        # x: (batch_size, input_size)
        x = self.input_proj(x)  # (batch_size, dim_feedforward)
        x = x.unsqueeze(1)  # (batch_size, seq_len=1, dim_feedforward)
        x = x.permute(1, 0, 2)  # (seq_len=1, batch_size, dim_feedforward)
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)  # (seq_len=1, batch_size, dim_feedforward)
        x = x.squeeze(0)  # (batch_size, dim_feedforward)
        z = self.output_proj(x)  # (batch_size, latent_size)
        return z

class TransformerDecoderRecon(nn.Module):
    def __init__(self, latent_size, output_size, num_layers=2, nhead=4, dim_feedforward=256, dropout=0.1):
        super(TransformerDecoderRecon, self).__init__()
        self.input_proj = nn.Linear(latent_size, dim_feedforward)
        self.positional_encoding = PositionalEncoding(dim_feedforward)
        decoder_layer = nn.TransformerDecoderLayer(d_model=dim_feedforward, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.output_proj = nn.Linear(dim_feedforward, output_size)

    def forward(self, z):
        # z: (batch_size, latent_size)
        z = self.input_proj(z)  # (batch_size, dim_feedforward)
        z = z.unsqueeze(1)  # (batch_size, seq_len=1, dim_feedforward)
        z = z.permute(1, 0, 2)  # (seq_len=1, batch_size, dim_feedforward)
        z = self.positional_encoding(z)
        # Use z as both memory and target for simplicity
        y = self.transformer_decoder(z, z)  # (seq_len=1, batch_size, dim_feedforward)
        y = y.squeeze(0)  # (batch_size, dim_feedforward)
        x_recon = self.output_proj(y)  # (batch_size, output_size)
        return x_recon

class TransformerDecoderPred(nn.Module):
    def __init__(self, latent_size, output_size, num_layers=2, nhead=4, dim_feedforward=256, dropout=0.1):
        super(TransformerDecoderPred, self).__init__()
        self.input_proj = nn.Linear(latent_size, dim_feedforward)
        self.positional_encoding = PositionalEncoding(dim_feedforward)
        decoder_layer = nn.TransformerDecoderLayer(d_model=dim_feedforward, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.output_proj = nn.Linear(dim_feedforward, output_size)

    def forward(self, z):
        # z: (batch_size, latent_size)
        z = self.input_proj(z)  # (batch_size, dim_feedforward)
        z = z.unsqueeze(1)  # (batch_size, seq_len=1, dim_feedforward)
        z = z.permute(1, 0, 2)  # (seq_len=1, batch_size, dim_feedforward)
        z = self.positional_encoding(z)
        # Use z as both memory and target for simplicity
        y = self.transformer_decoder(z, z)  # (seq_len=1, batch_size, dim_feedforward)
        y = y.squeeze(0)  # (batch_size, dim_feedforward)
        y_pred = self.output_proj(y)  # (batch_size, output_size)
        return y_pred

# 10. 定义数据集类
class UnifiedDataset(Dataset):
    def __init__(self, data_list, input_size, mean_values):
        """
        初始化数据集。

        参数:
        - data_list (list of dict): 每个字典应包含 'input_data', 'target_data', 'input_length', 'level_label'。
        - input_size (int): 最大输入大小，用于填充。
        - mean_values (dict): 每个层级输入数据的均值，用于填充。
        """
        self.samples = []
        self.input_size = input_size
        self.mean_values = mean_values  # 用于填充
        for data_dict in data_list:
            input_sequences, targets, levels = self.prepare_level_samples(
                data_dict['input_data'],
                data_dict['target_data'],
                data_dict['input_length'],
                data_dict['level_label']
            )
            self.samples.extend(zip(input_sequences, targets, levels))
        # 如果有样本，则解包
        if self.samples:
            self.inputs, self.targets, self.levels = zip(*self.samples)
            self.inputs = np.array(self.inputs)
            self.targets = np.array(self.targets)
            self.levels = np.array(self.levels)
        else:
            self.inputs = np.array([])
            self.targets = np.array([])
            self.levels = np.array([])

    def prepare_level_samples(self, input_data, target_data, input_length, level_label):
        """
        准备输入序列和对应的目标标签，基于时间对齐，并进行填充。

        参数:
        - input_data (pd.DataFrame): 当前层级的输入数据。
        - target_data (pd.DataFrame): 当前层级的目标数据。
        - input_length (int): 输入序列的长度。
        - level_label (str): 层级标签（A, B, C, D）。

        返回:
        - input_sequences (list of np.array): 输入序列列表。
        - targets (list of np.array): 目标标签列表。
        - levels (list of str): 层级标签列表。
        """
        input_data_sorted = input_data.sort_index()
        target_data_sorted = target_data.sort_index()

        input_sequences = []
        targets = []
        levels = []

        for target_date in target_data_sorted.index:
            # 获取 input_length 个输入样本，结束于 target_date 前一时间步
            if level_label == 'A':
                # Level A: daily_max, input_data is all_data (half-hourly)
                input_end = target_date - pd.Timedelta(minutes=30)
                input_start = input_end - pd.Timedelta(minutes=30 * input_length) + pd.Timedelta(minutes=30)
            elif level_label == 'B':
                # Level B: weekly_max, input_data is daily_max
                input_end = target_date - pd.Timedelta(days=1)
                input_start = input_end - pd.Timedelta(days=1 * input_length) + pd.Timedelta(days=1)
            elif level_label == 'C':
                # Level C: monthly_max, input_data is weekly_max
                input_end = target_date - pd.Timedelta(weeks=1)
                input_start = input_end - pd.Timedelta(weeks=1 * input_length) + pd.Timedelta(weeks=1)
            elif level_label == 'D':
                # Level D: yearly_max, input_data is monthly_max
                input_end = target_date - pd.offsets.MonthBegin(1)
                input_start = input_end - pd.offsets.MonthBegin(input_length) + pd.offsets.MonthBegin(1)
            else:
                print(f"未知的层级标签: {level_label}")
                continue

            try:
                input_seq = input_data_sorted.loc[input_start:input_end].values
                actual_length = len(input_seq)
                if actual_length < input_length:
                    pad_length = input_length - actual_length
                    pad_values = self.mean_values[level_label]
                    pad_array = np.tile(pad_values, pad_length).reshape(pad_length, -1)
                    input_seq = np.vstack([input_seq, pad_array])
                elif actual_length > input_length:
                    input_seq = input_seq[-input_length:]
            except KeyError:
                print(f"层级 {level_label} 日期 {target_date} 的输入数据不足。跳过。")
                continue

            # 展平并添加层级编码
            input_with_level = np.concatenate([input_seq.flatten(), level_encodings[level_label]])

            # 填充或截断以达到最大输入大小
            if len(input_with_level) < self.input_size:
                padding_size = self.input_size - len(input_with_level)
                padding = np.zeros(padding_size, dtype=np.float32)
                input_with_level = np.concatenate([input_with_level, padding])
            elif len(input_with_level) > self.input_size:
                input_with_level = input_with_level[:self.input_size]

            # 获取目标标签
            target = target_data_sorted.loc[target_date].values

            # 检查目标标签是否包含NaN
            if np.isnan(target).any():
                print(f"层级 {level_label} 日期 {target_date} 的目标标签包含NaN。跳过。")
                continue

            input_sequences.append(input_with_level)
            targets.append(target)
            levels.append(level_label)

        return input_sequences, targets, levels

# 11. 为每个层级准备数据
data_list = []

# Level A
data_list.append({
    'input_data': scaled_all_data,
    'target_data': scaled_daily_max,
    'input_length': 48,  # 48个半小时
    'level_label': 'A'
})

# Level B
data_list.append({
    'input_data': scaled_daily_max,
    'target_data': scaled_weekly_max,
    'input_length': 7,  # 7天
    'level_label': 'B'
})

# Level C
data_list.append({
    'input_data': scaled_weekly_max,
    'target_data': scaled_monthly_max,
    'input_length': 4,  # 4周
    'level_label': 'C'
})

# Level D
data_list.append({
    'input_data': scaled_monthly_max,
    'target_data': scaled_yearly_max,
    'input_length': 12,  # 12个月
    'level_label': 'D'
})

# 12. 创建统一数据集
dataset_full = UnifiedDataset(data_list, input_size, mean_values)
# print(f"统一数据集创建完成。总样本数量: {len(dataset_full)}")

# # 13. 检查每个层级的样本数量
# if len(dataset_full) > 0:
#     level_counts = Counter(dataset_full.levels)
#     print(f"每个层级的样本数量: {level_counts}")
# else:
#     print("没有生成任何样本，请检查数据和参数设置。")

# 14. 按层级划分训练集和测试集
def split_dataset(dataset, train_ratio=0.8):
    """
    按层级划分训练集和测试集，确保每个层级的样本按比例分配。

    参数:
    - dataset (UnifiedDataset): 统一数据集。
    - train_ratio (float): 训练集比例（默认为0.8）。

    返回:
    - (tuple): (训练集输入, 训练集标签, 训练集层级), (测试集输入, 测试集标签, 测试集层级)
    """
    train_samples = []
    test_samples = []
    levels = dataset.levels
    unique_levels = set(levels)
    for level in unique_levels:
        indices = np.where(levels == level)[0]
        if len(indices) == 0:
            continue
        np.random.shuffle(indices)  # 打乱每个层级的索引
        n_train = int(len(indices) * train_ratio)
        train_indices = indices[:n_train]
        test_indices = indices[n_train:]
        train_samples.extend([dataset.samples[i] for i in train_indices])
        test_samples.extend([dataset.samples[i] for i in test_indices])
    # 解包训练集和测试集
    if train_samples:
        train_inputs, train_targets, train_levels = zip(*train_samples)
        train_inputs = np.array(train_inputs)
        train_targets = np.array(train_targets)
        train_levels = np.array(train_levels)
    else:
        train_inputs, train_targets, train_levels = np.array([]), np.array([]), np.array([])
    if test_samples:
        test_inputs, test_targets, test_levels = zip(*test_samples)
        test_inputs = np.array(test_inputs)
        test_targets = np.array(test_targets)
        test_levels = np.array(test_levels)
    else:
        test_inputs, test_targets, test_levels = np.array([]), np.array([]), np.array([])
    return (train_inputs, train_targets, train_levels), (test_inputs, test_targets, test_levels)

# 进行数据拆分
(train_inputs, train_targets, train_levels), (test_inputs, test_targets, test_levels) = split_dataset(dataset_full, train_ratio=0.8)
print(f"训练集样本数量: {len(train_inputs)}, 测试集样本数量: {len(test_inputs)}")

# 15. 定义自定义数据集类用于训练和测试
class CustomDataset(Dataset):
    def __init__(self, inputs, targets, levels):
        self.inputs = inputs
        self.targets = targets
        self.levels = levels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        x = torch.tensor(self.inputs[idx], dtype=torch.float32)
        y = torch.tensor(self.targets[idx], dtype=torch.float32)
        level = self.levels[idx]
        return x, y, level

train_dataset = CustomDataset(train_inputs, train_targets, train_levels)
test_dataset = CustomDataset(test_inputs, test_targets, test_levels)

# 16. 创建DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("DataLoaders创建完成。")

# 17. 定义基于 Transformer 的 Encoder 和 Decoders（替换原有的 MLP 模型）
# Transformer 已经在上方定义为 TransformerEncoderModel, TransformerDecoderRecon, TransformerDecoderPred

# 18. 实例化模型
hidden_size = 128  # 可以调整为适合 Transformer 的隐藏维度
latent_size = 64
output_size = len(all_columns)  # 用于预测的特征数量

# 实例化基于 Transformer 的模型
encoder = TransformerEncoderModel(input_size=input_size, latent_size=latent_size, num_layers=2, nhead=4, dim_feedforward=256, dropout=0.1)
decoder_recon = TransformerDecoderRecon(latent_size=latent_size, output_size=input_size - num_levels, num_layers=2, nhead=4, dim_feedforward=256, dropout=0.1)
decoder_pred = TransformerDecoderPred(latent_size=latent_size, output_size=output_size, num_layers=2, nhead=4, dim_feedforward=256, dropout=0.1)

print("基于 Transformer 的模型实例化完成。")

# 19. 定义损失函数和优化器
criterion_recon = nn.MSELoss()
criterion_pred = nn.MSELoss()
optimizer_encoder = optim.Adam(list(encoder.parameters()) + list(decoder_recon.parameters()), lr=0.001)
optimizer_decoder = optim.Adam(decoder_pred.parameters(), lr=0.001)

print("损失函数和优化器定义完成。")

# 20. 定义训练编码器的函数
def train_encoder(encoder, decoder_recon, dataloader, criterion, optimizer, num_epochs=50):
    """
    使用重建损失训练编码器和重建解码器。

    参数:
    - encoder (nn.Module): 编码器模型。
    - decoder_recon (nn.Module): 重建解码器模型。
    - dataloader (DataLoader): 训练数据加载器。
    - criterion (nn.Module): 损失函数。
    - optimizer (optim.Optimizer): 优化器。
    - num_epochs (int): 训练轮数。
    """
    encoder.train()
    decoder_recon.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for x_batch, _, _ in dataloader:
            optimizer.zero_grad()
            # 前向传播
            z = encoder(x_batch)
            x_recon = decoder_recon(z)
            # 排除层级编码部分
            x_original = x_batch[:, :-num_levels]
            loss = criterion(x_recon, x_original)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Encoder Loss: {avg_loss:.6f}')

# 21. 训练编码器
print("开始训练编码器...")
train_encoder(encoder, decoder_recon, train_loader, criterion_recon, optimizer_encoder, num_epochs=50)
print("编码器训练完成。")

# 22. 冻结编码器参数
for param in encoder.parameters():
    param.requires_grad = False
print("编码器参数已冻结。")

# 23. 定义训练解码器的函数
def train_decoder(decoder_pred, encoder, dataloader, criterion, optimizer, num_epochs=10):
    """
    使用预测损失训练解码器。

    参数:
    - decoder_pred (nn.Module): 预测解码器模型。
    - encoder (nn.Module): 编码器模型（已冻结）。
    - dataloader (DataLoader): 训练数据加载器。
    - criterion (nn.Module): 损失函数。
    - optimizer (optim.Optimizer): 优化器。
    - num_epochs (int): 训练轮数。
    """
    decoder_pred.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for x_batch, y_batch, _ in dataloader:
            optimizer.zero_grad()
            # 前向传播
            z = encoder(x_batch)
            y_pred = decoder_pred(z)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Decoder Loss: {avg_loss:.6f}')

# 24. 训练解码器
print("开始训练解码器...")
train_decoder(decoder_pred, encoder, train_loader, criterion_pred, optimizer_decoder, num_epochs=30)
print("解码器训练完成。")

# 25. 定义测试模型的函数
def test_model(encoder, decoder_pred, dataloader, criterion):
    """
    测试模型并收集预测结果和实际标签。

    参数:
    - encoder (nn.Module): 编码器模型。
    - decoder_pred (nn.Module): 预测解码器模型。
    - dataloader (DataLoader): 测试数据加载器。
    - criterion (nn.Module): 损失函数。

    返回:
    - all_preds (np.array): 所有预测结果。
    - all_targets (np.array): 所有实际标签。
    - all_levels (list): 所有样本的层级标签。
    """
    encoder.eval()
    decoder_pred.eval()
    total_loss = 0
    all_preds = []
    all_targets = []
    all_levels = []
    with torch.no_grad():
        for x_batch, y_batch, levels in dataloader:
            z = encoder(x_batch)
            y_pred = decoder_pred(z)
            loss = criterion(y_pred, y_batch)
            total_loss += loss.item()
            all_preds.append(y_pred.cpu().numpy())
            all_targets.append(y_batch.cpu().numpy())
            all_levels.extend(levels)
    avg_loss = total_loss / len(dataloader)
    print(f'Test Loss: {avg_loss:.6f}')
    # 拼接所有预测和实际标签
    all_preds = np.vstack(all_preds)
    all_targets = np.vstack(all_targets)
    return all_preds, all_targets, all_levels

# 26. 测试模型
print("开始测试模型...")
all_preds_test, all_targets_test, all_levels_test = test_model(encoder, decoder_pred, test_loader, criterion_pred)
print("模型测试完成。")

# 27. 逆归一化预测结果和实际标签
# 确保预测结果和标签的形状与 scaler 一致
try:
    all_preds_original = scaler.inverse_transform(all_preds_test)
    all_targets_original = scaler.inverse_transform(all_targets_test)
    print("逆归一化完成。")
except Exception as e:
    print(f"逆归一化时出错: {e}")
    all_preds_original = all_preds_test
    all_targets_original = all_targets_test

# 28. 检查标签是否全为零
print("检查实际标签是否全为零...")
actual_zero = np.sum(all_targets_original == 0)
print(f"实际标签中值为零的数量: {actual_zero} (总标签数量: {all_targets_original.size})")

# 29. 创建结果 DataFrame 并保存
# 使用频道的序号（从0开始）代替实际的频道名称
results_df = pd.DataFrame({
    'Level': all_levels_test,
})

# 为每个通道添加预测值和实际值，使用序号代替频道名称
for i in range(len(all_columns)):
    results_df[f'Predicted_{i}'] = all_preds_original[:, i]
    results_df[f'Actual_{i}'] = all_targets_original[:, i]

# 创建输出目录（如果不存在）
output_dir = 'prediction_results'
os.makedirs(output_dir, exist_ok=True)

# 保存所有测试结果到一个CSV文件
results_df.to_csv(os.path.join(output_dir, 'prediction_results_test.csv'), index=False)
print("所有测试结果已保存到 'prediction_results_test.csv'。")

# 30. 按层级保存结果
unique_levels = sorted(set(all_levels_test))
for level in unique_levels:
    level_df = results_df[results_df['Level'] == level]
    filename = f'prediction_results_{level}_test.csv'
    level_df.to_csv(os.path.join(output_dir, filename), index=False)
    print(f"层级 {level} 的测试结果已保存到 '{filename}'。")

# 31. 打印部分结果以供验证
print("示例结果：")
print(results_df.head())

# 32. 保存标签和预测值的样本以供进一步检查
labels_output_path = os.path.join(output_dir, 'actual_labels_sample.csv')
actual_labels_sample = results_df[['Level'] + [f'Actual_{i}' for i in range(len(all_columns))]].head(100)
actual_labels_sample.to_csv(labels_output_path, index=False)
print(f"实际标签示例已保存到 '{labels_output_path}'。")

predictions_output_path = os.path.join(output_dir, 'predicted_values_sample.csv')
predicted_values_sample = results_df[['Level'] + [f'Predicted_{i}' for i in range(len(all_columns))]].head(100)
predicted_values_sample.to_csv(predictions_output_path, index=False)
print(f"预测结果示例已保存到 '{predictions_output_path}'。")

# 33. 可视化部分预测结果与实际值
def plot_predictions(actual, predicted, channel_num, output_dir, num_samples=100):
    """
    绘制实际值与预测值的对比图，并保存为PNG文件。

    参数:
    - actual (np.array): 实际值。
    - predicted (np.array): 预测值。
    - channel_num (int): 频道的序号编号。
    - output_dir (str): 输出目录。
    - num_samples (int): 绘制的样本数量。
    """
    plt.figure(figsize=(15, 5))
    plt.plot(actual[:num_samples], label='Actual')
    plt.plot(predicted[:num_samples], label='Predicted')
    plt.title(f'Channel: {channel_num} - Actual vs Predicted')
    plt.xlabel('Sample')
    plt.ylabel('Value')
    plt.legend()
    plt.savefig(os.path.join(output_dir, f'plot_{channel_num}.png'))
    plt.close()

# 绘制前5个通道的预测结果，使用顺序编号从0开始
for i in range(5):
    if i < len(all_columns):
        plot_predictions(all_targets_original[:, i], all_preds_original[:, i], i, output_dir)
        print(f"预测与实际值对比图已保存为 'plot_{i}.png'。")
    else:
        print(f"频道索引 {i} 超出范围。")


所有数据已加载。样本数量: 87695, 通道数量: 314
聚合数据集已创建。
daily_max 样本数量: 1827
weekly_max 样本数量: 262
monthly_max 样本数量: 60
yearly_max 样本数量: 5
数据归一化完成。
确定的最大输入大小: 15076
训练集样本数量: 1722, 测试集样本数量: 432
DataLoaders创建完成。
基于 Transformer 的模型实例化完成。
损失函数和优化器定义完成。
开始训练编码器...
Epoch [1/50], Encoder Loss: 0.062679
Epoch [10/50], Encoder Loss: 0.006580
Epoch [20/50], Encoder Loss: 0.004119
Epoch [30/50], Encoder Loss: 0.003463
Epoch [40/50], Encoder Loss: 0.003051
Epoch [50/50], Encoder Loss: 0.002733
编码器训练完成。
编码器参数已冻结。
开始训练解码器...
Epoch [1/30], Decoder Loss: 0.064601
Epoch [5/30], Decoder Loss: 0.020835
Epoch [10/30], Decoder Loss: 0.016509
Epoch [15/30], Decoder Loss: 0.015229
Epoch [20/30], Decoder Loss: 0.013791
Epoch [25/30], Decoder Loss: 0.013344
Epoch [30/30], Decoder Loss: 0.013017
解码器训练完成。
开始测试模型...
Test Loss: 0.012237
模型测试完成。
逆归一化完成。
检查实际标签是否全为零...
实际标签中值为零的数量: 12677 (总标签数量: 135648)


/var/folders/47/sjt2v1051ll309z65l5f0k380000gn/T/ipykernel_27221/3040476615.py:619: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results_df[f'Actual_{i}'] = all_targets_original[:, i]
/var/folders/47/sjt2v1051ll309z65l5f0k380000gn/T/ipykernel_27221/3040476615.py:618: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results_df[f'Predicted_{i}'] = all_preds_original[:, i]
/var/folders/47/sjt2v1051ll309z65l5f0k380000gn/T/ipykernel_27221/3040476615.py:619: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

所有测试结果已保存到 'prediction_results_test.csv'。
层级 A 的测试结果已保存到 'prediction_results_A_test.csv'。
层级 B 的测试结果已保存到 'prediction_results_B_test.csv'。
层级 C 的测试结果已保存到 'prediction_results_C_test.csv'。
层级 D 的测试结果已保存到 'prediction_results_D_test.csv'。
示例结果：
  Level   Predicted_0  Actual_0   Predicted_1  Actual_1   Predicted_2  \
0     C  10189.010742    9914.0   9861.779297    9793.0  19857.130859   
1     C  11002.370117   11285.0  10491.099609   10390.0  15400.275391   
2     C  10624.606445   10626.0  10058.803711    9745.0  15600.765625   
3     C  11227.880859   12457.0  10051.225586   10224.0  26327.597656   
4     C  10266.386719   10090.0   9923.062500    9930.0  21408.517578   

   Actual_2   Predicted_3  Actual_3   Predicted_4  ...  Predicted_309  \
0   18728.0  13127.419922   12667.0  18867.169922  ...    2873.083008   
1   12716.0  14422.903320   13640.0  14702.250000  ...    2270.162354   
2   15434.0  14455.497070   14682.0  16111.846680  ...    2738.566406   
3   28088.0  13883.872070   1